In [4]:
# import libraries

import os
import uuid
import pickle

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [1]:
year = 2021
month = 2
taxi_type = 'fhv'

input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv('RUN_ID', 'c4ec5ceed3004e4e81021e729bca448c')

In [5]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PUlocationID'] + '_' + df['DOlocationID']

    categorical = ['PU_DO']
    dicts = df[categorical].to_dict(orient='records')
    return dicts

In [4]:
def load_model(run_id):
    logged_model = f's3://bhagabat-fhv-taxi-prediction/1/{RUN_ID}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['pickup_datetime'] = df['pickup_datetime']
    df_result['dropOff_datetime'] = df['dropOff_datetime']
    df_result['PUlocationID'] = df['PUlocationID']
    df_result['DOlocationID'] = df['DOlocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)
    print(f'Output file saved: {output_file}')

In [5]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)


2022/06/30 05:55:51 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.1.0, required: cloudpickle==2.0.0)
 - psutil (current: 5.9.1, required: psutil==5.8.0)
 - typing-extensions (current: 4.2.0, required: typing-extensions==4.1.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Output file saved: output/fhv/2021-02.parquet


In [ ]:
mlflow.pyfunc.get_model_dependencies(model_uri)